<a href="https://colab.research.google.com/github/nenecchiCV/internal-competition-2024/blob/main/method2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
api_key = "b846546e2316d4194c74a497599a7b83c9127f07"

In [ ]:
!pip install colorama
!pip install wandb -qqq
import wandb

try:
    wandb.login(key=api_key)
    anonymous = None
except:
    anonymous = "must"
    print('To use your W&B account,\nGo to Add-ons -> Secrets and provide your W&B access token. Use the Label name as WANDB. \nGet your W&B access token from here: https://wandb.ai/authorize')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.8/258.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.1 MB/s eta 0:00:00


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import pandas as pd
import numpy as np
data1_df = pd.read_csv('/content/drive/MyDrive/daicc2024/data_1.csv')
data2_df = pd.read_csv('/content/drive/MyDrive/daicc2024/data_2.csv')
data_df = pd.concat([data1_df, data2_df], axis=0)
data_df.head()

test_df = pd.read_csv('/content/drive/MyDrive/daicc2024/test.csv')
all_df = pd.concat([data_df, test_df], axis=0)

In [ ]:
X1 = data_df.drop(columns=["id",'parameter',"spec"])
x2 = data_df[['spec']]
y = data_df[['parameter']]

X1_all = all_df.drop(columns=["id",'parameter',"spec"])
x2_min_max = np.array([[90], [110]]).astype('float')
x2_min_max_df = pd.DataFrame(data=x2_min_max, columns=["spec"], dtype=float)

from sklearn.preprocessing import StandardScaler, MinMaxScaler
import torch

# trainというDataFrameにfit
X1_scaler = StandardScaler().fit(X1_all)
x2_scaler = MinMaxScaler(feature_range=(-1, 1)).fit(x2_min_max_df)
X1_scaled = X1_scaler.transform(X1)
x2_scaled = x2_scaler.transform(x2)

X1_scaled_tensor = torch.tensor(X1_scaled.reshape(-1, 3, 5), dtype=torch.float64)
x2_scaled_tensor = torch.tensor(x2_scaled, dtype=torch.float64)
y_tensor = torch.tensor(y.values, dtype=torch.float64)

X1_test = test_df.drop(columns=["id"])
test_ids = test_df["id"]
X1_scaled_test = X1_scaler.transform(X1_test)
X1_scaled_test_tensor = torch.tensor(X1_scaled_test.reshape(-1, 3, 5), dtype=torch.float64)

メモ：計測値とスコアをX、パラメータをYとして学習→結果を出すときはスコアを100にする

RNN
https://qiita.com/MENDY/items/99da56f61f9af51dda15

In [ ]:
from torch.utils import data as data

class MyDataset(data.Dataset):
    def __init__(self, X1_scaled_tensor, x2_scaled_tensor, y_tensor):
        self.X1 = X1_scaled_tensor
        self.x2 = x2_scaled_tensor
        self.y = y_tensor

    def __getitem__(self, index):
        return self.X1[index], self.x2[index], self.y[index]

    def __len__(self):
        return len(self.y)

In [ ]:
# import torch_xla
# import torch_xla.core.xla_model as xm

class CFG:
    seed = 99
    exp_name = 'exp_1'
    comment = exp_name
    model_name = 'LSTM-FC'
    train_bs = 8
    valid_bs = train_bs
    data_size = [[3, 5], [1]]
    epochs = 50
    lr = 5e-4
    scheduler = 'CosineAnnealingLR'
    min_lr = 1e-5
    T_max = int(30000 / train_bs * epochs) + 50
    T_0 = 25
    warmup_epochs = 0
    wd = 1e-6
    n_fold = 5
    folds = []
    device = "cpu"
    # device = xm.xla_device()
    lstm_input_size=5
    lstm_hidden_size=10
    fc_hidden_size=128
    dropout_rate = 0.4
#     target_dataset='uwmgi-stride2-dataset/images'

In [ ]:
from torch import nn
from torch.nn import functional as F
class LSTMScalarPredictor(nn.Module):
    def __init__(self, lstm_input_size=CFG.lstm_input_size, lstm_hidden_size=CFG.lstm_hidden_size, fc_hidden_size=CFG.fc_hidden_size):
        super(LSTMScalarPredictor, self).__init__()
        self.lstm_hidden_size = lstm_hidden_size
        self.lstm = nn.LSTM(lstm_input_size, lstm_hidden_size, batch_first=True)
        self.fc1 = nn.Linear(lstm_hidden_size + 1, fc_hidden_size)  # +1 for scalar input
        self.fc2 = nn.Linear(fc_hidden_size, fc_hidden_size)
        self.fc3 = nn.Linear(fc_hidden_size, 1)
        self.dropout1 = nn.Dropout2d(CFG.dropout_rate)
        self.dropout2 = nn.Dropout2d(CFG.dropout_rate)
        self.dropout3 = nn.Dropout2d(CFG.dropout_rate)
        self.bn1 = nn.BatchNorm1d(fc_hidden_size)
        self.bn2 = nn.BatchNorm1d(fc_hidden_size)

    def forward(self, X1, x2):
        # Initialize hidden state with zeros
        h0 = torch.zeros(1, X1.size(0), self.lstm_hidden_size).to(X1.device)
        c0 = torch.zeros(1, X1.size(0), self.lstm_hidden_size).to(X1.device)

        # Forward pass through LSTM layer
        lstm_H, _ = self.lstm(X1, (h0, c0))

        # Only take the output from the last time step
        x = torch.cat([lstm_H[:, -1, :], x2], dim=1)
        x = self.dropout1(x)
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.dropout2(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.dropout3(x)
        out = self.fc3(x)

        return out

    def init_wandb(self):
      for name, param in self.named_parameters():
        if 'lstm.weight' in name:
            nn.init.xavier_uniform_(param)  # LSTM層の重みをXavier初期化
        elif 'fc3.weight' in name:
            nn.init.xavier_uniform_(param)  # FC-softmax層の重みをXavier初期化
        elif 'fc.weight' in name:
            nn.init.kaiming_normal_(param, nonlinearity='relu')  # FC層-reluの重みをHe初期化


In [ ]:
MSELoss = nn.MSELoss()
def criterion(y_pred, y_true):
    return MSELoss(y_pred, y_true)

In [ ]:
import random
import os
def set_seed(seed = 42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    print('> SEEDING DONE')

set_seed(CFG.seed)

> SEEDING DONE


In [ ]:
from tqdm import tqdm
import gc

def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()

    dataset_size = 0
    running_loss = 0.0

    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc='Train ')
    for step, (X1, x2, y_true) in pbar:
        X1 = X1.to(device, dtype=torch.float)
        x2  = x2.to(device, dtype=torch.float)
        y_true  = y_true.to(device, dtype=torch.float)

        batch_size = X1.size(0)

        optimizer.zero_grad()

        y_pred = model(X1, x2)
        loss = criterion(y_pred, y_true)
        loss.backward()

        optimizer.step()

        if scheduler is not None:
            scheduler.step()

        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size

        epoch_loss = running_loss / dataset_size

        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        current_lr = optimizer.param_groups[0]['lr']
        pbar.set_postfix(train_loss=f'{epoch_loss:0.4f}',
                        lr=f'{current_lr:0.5f}',
                        gpu_mem=f'{mem:0.2f} GB')

    torch.cuda.empty_cache()
    gc.collect()

    return epoch_loss

@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()

    dataset_size = 0
    running_loss = 0.0

    val_scores = []

    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc='Valid ')
    for step, (X1, x2, y_true) in pbar:
        X1 = X1.to(device, dtype=torch.float)
        x2 = x2.to(device, dtype=torch.float)
        y_true  = y_true.to(device, dtype=torch.float)

        batch_size = X1.size(0)

        y_pred = model(X1, x2)
        loss = criterion(y_pred, y_true)

        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size

        epoch_loss = running_loss / dataset_size

        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        current_lr = optimizer.param_groups[0]['lr']
        pbar.set_postfix(valid_loss=f'{epoch_loss:0.4f}',
                        lr=f'{current_lr:0.5f}',
                        gpu_memory=f'{mem:0.2f} GB')

    torch.cuda.empty_cache()
    gc.collect()

    return epoch_loss

In [ ]:
import copy
from collections import defaultdict
import time
from colorama import Fore, Back, Style
c_  = Fore.GREEN
sr_ = Style.RESET_ALL

def run_training(model, optimizer, scheduler, device, num_epochs):
    # To automatically log gradients
    run.watch(model, log_freq=100)

    start = time.time()

    if torch.cuda.is_available():
        print("cuda: {}\n".format(torch.cuda.get_device_name()))

    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss      = np.inf
    best_epoch     = -1
    history = defaultdict(list)

    for epoch in range(1, num_epochs + 1):
        gc.collect()
        print(f'Epoch {epoch}/{num_epochs}', end='')
        train_loss = train_one_epoch(model, optimizer, scheduler,
                                           dataloader=train_loader,
                                           device=CFG.device, epoch=epoch)

        val_loss = valid_one_epoch(model, valid_loader,
                                                 device=CFG.device,
                                                 epoch=epoch)

        history['Train Loss'].append(train_loss)
        history['Valid Loss'].append(val_loss)

        # Log the metrics
        run.log({"Train Loss": train_loss,
                   "Valid Loss": val_loss,
                   "LR":scheduler.get_last_lr()[0]})

        if val_loss < best_loss:
            print(f"{c_}Valid Score Improved ({best_loss:0.4f} ---> {val_loss:0.4f})")
            best_loss = val_loss
            best_epoch   = epoch
            run.summary["Best Loss"]    = best_loss
            run.summary["Best Epoch"]   = best_epoch
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = f"/content/drive/MyDrive/daicc2024/output/best_epoch-{fold:02d}.bin"
            torch.save(model.state_dict(), PATH)
            # Save a model file from the current directory
            run.save(PATH)
            print(f"Model Saved{sr_}")


        print(); print()

    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best Loss: {:.4f}".format(best_loss))

    return model, history

In [ ]:
def fetch_scheduler(optimizer):
    if CFG.scheduler == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=CFG.T_max,
                                                   eta_min=CFG.min_lr)
    elif CFG.scheduler == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=CFG.T_0,
                                                             eta_min=CFG.min_lr)
    elif CFG.scheduler == 'ReduceLROnPlateau':
        scheduler = lr_scheduler.ReduceLROnPlateau(optimizer,
                                                   mode='min',
                                                   factor=0.1,
                                                   patience=7,
                                                   threshold=0.0001,
                                                   min_lr=CFG.min_lr,)
    elif CFG.scheduer == 'ExponentialLR':
        scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.85)
    elif CFG.scheduler == None:
        return None

    return scheduler

In [ ]:
from sklearn.model_selection import KFold
import torch.optim as optim
from torch.optim import lr_scheduler

batch_size = CFG.train_bs

my_dataset = MyDataset(X1_scaled_tensor, x2_scaled_tensor, y_tensor)
kfold = KFold(n_splits=CFG.n_fold, random_state=99, shuffle=True)
model = LSTMScalarPredictor(CFG.lstm_input_size, CFG.lstm_hidden_size, CFG.fc_hidden_size)
# 重みとバイアスの初期化
model.init_wandb()

for fold, (train_ids, valid_ids) in enumerate(kfold.split(my_dataset)):
    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')

    # Sample elements randomly from a given list of ids, no replacement.
    train_dataset = torch.utils.data.dataset.Subset(my_dataset, train_ids)
    valid_dataset = torch.utils.data.dataset.Subset(my_dataset, valid_ids)

    # Define data loaders for training and testing data in this fold
    train_loader = torch.utils.data.DataLoader(
                      train_dataset,
                      batch_size=batch_size)
    valid_loader = torch.utils.data.DataLoader(
                      valid_dataset,
                      batch_size=batch_size)

    run = wandb.init(project='internal-competition',
                      config={k:v for k, v in dict(vars(CFG)).items() if '__' not in k},
                      anonymous=anonymous,
                      name=f"fold-{fold}|model-{CFG.model_name}",
                      group=CFG.exp_name,
                    )


    optimizer = optim.Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.wd)
    scheduler = fetch_scheduler(optimizer)
    model, history = run_training(model, optimizer, scheduler,
                                  device=CFG.device,
                                  num_epochs=CFG.epochs)
    run.finish()

FOLD 0
--------------------------------


Epoch 1/50

Train :   0%|          | 0/2000 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:1345: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)
Valid : 100%|██████████| 500/500 [00:03<00:00, 143.02it/s, gpu_memory=0.00 GB, lr=0.00050, valid_loss=0.6488]
wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


Valid Score Improved (inf ---> 0.6488)
Model Saved


Epoch 2/50

Valid : 100%|██████████| 500/500 [00:03<00:00, 131.51it/s, gpu_memory=0.00 GB, lr=0.00050, valid_loss=0.6881]


...

Valid : 100%|██████████| 500/500 [00:03<00:00, 126.78it/s, gpu_memory=0.00 GB, lr=0.00050, valid_loss=0.5851]


Valid Score Improved (inf ---> 0.5851)
Model Saved


Epoch 2/50

Valid : 100%|██████████| 500/500 [00:03<00:00, 137.42it/s, gpu_memory=0.00 GB, lr=0.00050, valid_loss=0.5912]




Epoch 3/50

Train :  55%|█████▌    | 1106/2000 [00:17<00:15, 56.07it/s, gpu_mem=0.00 GB, lr=0.00050, train_loss=0.2049]

In [ ]:
class TestDataset(data.Dataset):
    def __init__(self, X1_scaled_tensor, x2_scaled_tensor):
        self.X1 = X1_scaled_tensor
        self.x2 = x2_scaled_tensor

    def __getitem__(self, index):
        return self.X1[index], self.x2[index]

    def __len__(self):
        return len(self.X1)

In [ ]:
X_test = X1_scaled_test_tensor
chpt_path = "/content/drive/MyDrive/daicc2024/output/best_epoch-00.bin"
model.load_state_dict(torch.load(chpt_path, map_location='cuda'))
model.eval()
target_spec_scaled = x2_scaler.transform(100)
x2_test  = torch.full((X_test.size(0), 1), target_spec_scaled).to(CFG.device, dtype=torch.float)
y_preds = np.zeros(X_test.size(0))

# Define data loaders for training and testing data in this fold
test_loader = torch.utils.data.DataLoader(
                  TestDataset(X_test, x2_test),
                  batch_size=1)
for idx, (X1, x2) in enumerate(tqdm(test_loader, total=len(test_loader), desc='Infer ')):
    y_pred = model(X1, x2)
    y_preds[idx] = y_pred

pred_df = pd.DataFrame({
    "id":test_ids,
    "parameter":y_preds
})
pred_df.to_csv(f'/content/drive/MyDrive/daicc2024/output/submission_{CFG.exp_name}.csv',index=False)

https://www.kaggle.com/code/fnands/uwmgi-levit-unet-baseline